### Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import warnings
import json

In [2]:
warnings.filterwarnings('ignore')

### Even Swaps Program

In [3]:
def error_messages (error_msg):
    """
    Function to print error messages in bold
    
    Parameters
    __________
    error_msg: error message to be printed
    
    Returns
    _______
    None
    """
    #For making text bold
    bold_start = "\033[1m" 
    bold_end = "\033[0m"
    print(bold_start + error_msg + bold_end)

In [4]:
def ranking_table (df):
    """
    Function to rank a consquences table
    User is asked to input rank for qualitative (non-numeric) objectives
    User is asked to input index for the ranks (which row the data is from)
    User is asked whether the ranking for each row is ascending or not
    
    Parameters
    __________
    df: dataframe to do the rankings on
    
    Returns
    _______
    df_ranked: ranked dataframe
    """
    
    print('\nConsequences table to be ranked')
    display(df)

    flag = False #For exiting program if user does not input number for the ranks

    dfwo = df.loc[:, df.columns[1:]] #df without objectives
    dfwo_copy = dfwo.copy()
    df_obj = df.iloc[:,[0]] #Objectives column

    rank_data = []

    for i in dfwo:
        df_str = dfwo[dfwo[i].apply(lambda x: isinstance(x, str))] #Checks whether df contains strings
        
    len_df_str = len(df_str.index) #Length of the index for df_str
    
    if df_str.empty:
        print('\nThere is no string objectives in the dataframe')
        
    else:
        print('\nDataframe with only string objectives')
        display(df_str)

        j = 0
        while j < len(df_str):
            for rowIndex, row in df_str.iterrows():
                    for columnIndex, value in row.items():
                        try:
                            input_rank = int(input('\nWhat would you like the rank for {} to be? '.format(value)))
                            flag = True
                        except ValueError:
                            value_error_msg = '\nError! That is not an Int number. Please enter an Int number\n'
                            error_messages(value_error_msg)
                            
                        if not flag:
                            return
                            
                        if (input_rank <= 0):
                            error_msg1 = '\nError! Your chosen rank is less than or' 
                            error_msg2 = ' equal to zero, but it has to be positive\n'
                            error_msg_whole = error_msg1 + error_msg2
                            error_messages(error_msg_whole)
                            return

                        rank_data.append(input_rank)
                    print()
                    j += 1
 
        splits = np.array_split(rank_data, len_df_str) #Split rank_data, based on len of index
        print('\nRanking data', rank_data)
        print('\nIt is important to remember that the position of the data should be')
        print('corresponding with the position of the data from the original CT')

        flag2 = False #For exiting program if user does not input number for the index position of where the ranks belong
        g = 0
        while g < len(splits):
            split_ranks = splits[g]
            try:
                input_rank_pos = int(input('\nWhich position in the dataframe is the data {} from? '.format(split_ranks)))
                dfwo_copy.at[input_rank_pos] = split_ranks
                flag2 = True
            except ValueError:
                value_error_msg = '\nError! That is not an Int number. Please enter an Int number\n'
                error_messages(value_error_msg)
            if not flag2:
                return
            
            if (input_rank_pos < 0):
                error_msg = '\nError! Your chosen index position is negative, but it has to be zero or greater\n'
                error_messages(error_msg)
                return

            g += 1
        
    print('\nDataframe with only numbers')
    display(dfwo_copy)
    print('******************')

    row_data = []

    for r in range(len(dfwo_copy)):
        row = dfwo_copy.loc[r]
        row_vals = row.values
        input_ascending = input('\nWould you like the ranking for {0} with index {1}'.format(row_vals, r) + 
                                ' to be Ascending or Not? A or N: ')

        if input_ascending.lower() in {'a'}:
            df_ranked_a = row.rank(method = 'min', ascending = True).astype(int)
            
        elif input_ascending.lower() in {'n'}:
            df_ranked_a = row.rank(method = 'min', ascending = False).astype(int)
        else:
            error_msg = 'Error! Options are A or N'
            error_messages(error_msg)
            break

        row_data.append(df_ranked_a)   

    df_ranked = pd.DataFrame(row_data)
    df_ranked = pd.concat([df_obj, df_ranked], axis = 1)
    
    print('\nRanked dataframe')
    display(df_ranked)

    return df_ranked

In [5]:
def cyo ():
    """
    Function for the CYO option in the user_choice(input_decision) function where the user
    is asked to create his/her own consequences table (CT)
    
    Parameters
    __________
    None
    
    Returns
    _______
    df: dataframe in CT format to be ranked
    """
    
    user_statement = """
    
    How to use the program to create consequences table
    1. Enter a number of objectives as data rows, e.g. 5
    2. Enter a number of alternatives, e.g. 5
    3. Enter header for alternatives like this: Alt1, Alt2 until however many alternatives you have
    4. Enter the name for the first objective, then the next until you have done so for all objectives
    5. Enter data for alternatives like this: 1700 1800

    Note: For step 5, it is important to remember that you enter data for alternatives 
    like this: data for alt 1 data for alt 2, and so on
    """

    print(user_statement)

    try:
        num_obj = int(input("\nEnter number of objectives (data rows): "))
        num_alts = int(input('\nEnter number of alternatives: '))

    except ValueError:
        value_error_msg = '\nError! That is not an Int number. Please enter an Int number\n'
        error_messages(value_error_msg)
        return

    obj_header = 'Objectives'

    alternatives_header = [y for y in input('\nEnter header for alternatives separated by comma: ').split(', ')]

    data_objectives = []
    data_alternatives = []

    p = 1 
    while p <= num_obj: 
        row_objectives = input('\nEnter name for objective {}: '.format(p))

        if row_objectives in data_objectives:
            error_msg = '\nThat objective name already exists'
            error_messages(error_msg)
            row_objectives = input('\nEnter a new name for objective {}: '.format(p))

        data_objectives.append(row_objectives)
        p += 1 

    q = 1

    input_text = '\nWhat type would you like to use for {}? Float, Int, or String (F, I, or S): '
    while q <= num_obj:
        input_type = input(input_text.format(data_objectives[q - 1]))

        if input_type.lower() in {'f'}:
            row_alternatives = [float(x) for x in input('\nEnter data for alternatives' +
                            ' for objective {} separated by space: '.format(data_objectives[q - 1])).split()]
        elif input_type.lower() in {'i'}:
            row_alternatives = [int(x) for x in input('\nEnter data for alternatives' +
                            ' for objective {} separated by space: '.format(data_objectives[q - 1])).split()]
        elif input_type.lower() in {'s'}:
            row_alternatives = [x for x in input('\nEnter data for alternatives' +
                            ' for objective {} separated by space: '.format(data_objectives[q - 1])).split()]
        else:
            error_msg = '\nError! Options are F, I, or S\n'
            error_messages(error_msg)
            return

        if len(row_alternatives) > num_alts:
            error_msg1 = ('\nError! No more than') + str(num_alts)
            error_msg2 = (' alternatives are allowed')
            error_msg_whole = error_msg1 + error_msg2
            error_messages(error_msg_whole)
            break

        data_alternatives.append(row_alternatives)
        q += 1

    df1 = pd.DataFrame(data_objectives, columns = [obj_header])
    df2 = pd.DataFrame(data_alternatives, columns = alternatives_header)

    con_df = pd.concat([df1, df2], axis = 1) #Consequences DF
    con_df.index = range(len(con_df.index))

    df = ranking_table(con_df) #Ranking consequences table

    return df #Returning ranking table of created consequences table

In [6]:
def lcf ():
    """
    Function for the LCF option in the user_choice(input_decision) function where the user
    is asked to load in a consequences table (CT) or a ranking table (RT)
    
    Parameters
    __________
    None
    
    Returns
    _______
    df: dataframe that is either in CT or RT format
    """
    
    flag = True #To exit program if file is not found
    flag2 = True #To exit program if user enters quit

    while flag2:
        try:
            file_decision = input('\nWould you like to load a consequences table' +
                                  ' or a ranking table (CT or RT)? Enter quit to quit: ')
            if file_decision.lower() == 'Quit':
                flag2 = False
                
        except EOFError:
            print('\nProgram Quit')
            break

        if (file_decision.lower() == 'rt'):
            try:
                filename = input('\nWhat is the name of the file you would like to load? Enter quit to quit: ')

                if (filename.lower() == 'quit'):
                    print('\nProgram Quit')
                    flag = False
                    
                else:
                    filename = filename + '.xlsx'
                    df = load_files(filename)
                    print('\nDisplaying loaded dataframe')
                    display(df)
                    return df #Returning loaded RT dataframe
                
            except FileNotFoundError:
                error_msg1 = ('\nError! ') + str(filename)
                error_msg2 = (' does not exist\n')
                error_msg_whole = error_msg1 + error_msg2
                error_messages(error_msg_whole)

        elif (file_decision.lower() == 'ct'):
            try:
                filename = input('\nWhat is the name of the file you would like to load? Enter quit to quit: ')

                if (filename.lower() == 'quit'):
                    print('\nProgram Quit')
                    flag = False

                else:
                    filename = filename + '.xlsx'
                    loaded_df = load_files(filename)
                    df = ranking_table(loaded_df)
                    return df #Returning loaded ranked CT dataframe
            
            except FileNotFoundError:
                error_msg1 = ('\nError! ') + str(filename)
                error_msg2 = (' does not exist\n')
                error_msg_whole = error_msg1 + error_msg2
                error_messages(error_msg_whole)

        elif (file_decision.lower() == 'quit'):
            print('\nProgram Quit')
            flag = False

        else:
            error_msg1 = ('\nError! ') + str(file_decision)
            error_msg2 = (' is not a valid option\n')
            error_msg_whole = error_msg1 + error_msg2
            error_messages(error_msg_whole)
            print('Options are CT or RT')

        if not flag:
            return

In [7]:
def user_choice (input_decision):
    """
    Function that lets the user create his/her own consequences table (CT) or load a previously created xlsx file
    by using the functions cyo and lcf, respectively 
    
    Parameters
    __________
    input_decision: Whether the user would like to create his/her own CT or load a previously created xlsx file
    
    Returns
    _______
    df: Dataframe to be used for even swaps
    """

    if (input_decision == 'CYO'): #Create Your Own
        df = cyo()
        return df
        
    elif (input_decision == 'LCF'): #Load Created File
        df = lcf()
        return df

    elif (input_decision.lower() == 'quit'):
            raise EOFError
    
    else:
        error_msg1 = ('\nError! ') + str(input_decision)
        error_msg2 = (' is not a valid option\n')
        error_msg_whole = error_msg1 + error_msg2
        error_messages(error_msg_whole)
        print('Options are CYO, or LCF')
        return

In [8]:
def equality_check (df):
    """
    Function to check if a dataframe row has equal values
    
    Parameters
    __________
    df: dataframe to do operations on
    
    Returns
    _______
    df with unique rows
    """
    
    df = df[df.nunique(axis = 1).ne(1)]
    return df 

In [9]:
def compare_alternatives (alt1, alt2):
    """
    Function that compares alternatives (columns) in a dataframe
    based on rank
    
    Parameters
    __________
    alt1: dataframe column for first alternative
    alt2: dataframe column for second alternative
    
    Returns
    _______
    dominated: loser between the alternatives, which alternative is being dominated by another
    dominating: winner amongst the alternatives, which alternative is dominating the other
    dominance_type: what type of dominance it is (practical or absolute)
    """
    
    alt1_wins = 0
    alt2_wins = 0
    
    dominated = 'none'
    dominating = 'none'
    dominance_type = 'no dominance'
    
    #Checks if alternative name is different from itself, 
    #since there is no reason to compare an alternative with itself
    if (alt1.name != alt2.name): 
        for row in range(0, len(alt1)):

            if (int(alt1[row]) < int(alt2[row])): #Updates alt 1 wins
                alt1_wins += 1
                
            elif (int(alt1[row]) > int(alt2[row])): #Updates alt 2 wins
                alt2_wins += 1        

        if (alt1_wins > alt2_wins & alt2_wins < 2): #Alt 1 dominates alt 2
            dominated = alt2.name
            dominating = alt1.name

            if (alt2_wins == 1):
                dominance_type = 'practical'
            elif (alt2_wins == 0):
                dominance_type = 'absolute'

        elif (alt2_wins > alt1_wins & alt1_wins < 2): #Alt 2 dominates alt 1
            dominated = alt1.name
            dominating = alt2.name

            if (alt1_wins == 1):
                dominance_type = 'practical'
            elif (alt1_wins == 0):
                dominance_type = 'absolute'
    
    return dominance_type, dominated, dominating

In [10]:
def almost_equal (df):
    """
    Function that checks whether the ranks of an alternative is one rank apart from another alternative 
    e.g. rank of alt 1 is 1 and rank of alt 2 is 2 for same objective, 
    would return these alternatives with the ranks for that objective
    
    Parameters
    __________
    df: dataframe to do operations on
    
    Returns
    _______
    None
    """
    
    dfwo = df.loc[:, df.columns[1:]] #df without objectives
    df_obj = df.iloc[:,0] # Dataframe objectives
    obj_index = df_obj.index.values
    difference = []
    
    for i in dfwo.columns:
        for j in dfwo.columns:
            alt1 = dfwo[i]
            alt2 = dfwo[j]
            name1 = alt1.name
            name2 = alt2.name

            if (name1 != name2):
                for count, (k, l) in enumerate(zip(alt1, alt2)):
                    diff = abs(k - l)
                    difference.append(diff)
                    if diff == 1:
                        if name1 < name2:
                            if count in obj_index:
                                obj_name = df_obj.at[count]
                                msg = ('Objective ' + obj_name + ' for ' 
                                        + name1 + ' and ' + name2 + ' is almost equal with these rankings: ' 
                                        + str(k) + ', ' + str(l))
                                print(msg)
                       
    if 1 not in difference:
        print('There are no ranks that are almost equal')

In [11]:
def count_num_objectives (alt1, alt2):
    """
    Function that counts the number of objectives that dominate another alternative
    
    Parameters
    __________
    alt1: dataframe column for first alternative
    alt2: dataframe column for second alternative
    
    Returns
    _______
    None
    """
    
    alt1_wins = 0
    alt2_wins = 0
    
    dominated = 'none'
    dominating = 'none'
    
    #Checks if alternative name is different from itself, 
    #since there is no reason to compare an alternative with itself
    if(alt1.name != alt2.name): 
        for row in range(0, len(alt1)):

            if (int(alt1[row]) < int(alt2[row])): #Updates alt 1 wins
                alt1_wins += 1
                
            elif (int(alt1[row]) > int(alt2[row])): #Updates alt 2 wins
                alt2_wins += 1
            
        if (alt1_wins > alt2_wins): #Alt 1 dominates alt 2
            dominated = alt2.name
            dominating = alt1.name
            print(dominating + ' is more likely to dominate ' + dominated)
        
        elif (alt1_wins == alt2_wins):
            if alt1.name > alt2.name:
                print('Neither ' + alt1.name + ' nor ' + alt2.name + ' is more likely to dominate the other')

In [12]:
def df_obj_count (df):
    """
    Function that uses the function count_num_objectives(alt1, alt2)
    to count how many objectives have a higher rank for alt1 compared with alt2
    
    Parameters
    __________
    df: dataframe to do operations on
    
    Returns
    _______
    None
    """
    for i in df.columns:
        for j in df.columns:
            count_num_objectives(df[i], df[j])

In [13]:
def max_rank (df):
    """
    Function that checks what the max rank is in a ranking table (RT)
    
    Parameters
    __________
    df: dataframe to do operations on
    
    Returns
    _______
    df_max_value: max rank in RT
    """
    
    dfwo = df.loc[:, df.columns[1:]] #Dataframe without objectives
    df_max_value = dfwo.max().max()
    return df_max_value

In [14]:
def even_swaps (df, max_df):
    """
    Function that does even swaps on a dataframe
    The user inputs which objective and alternative he/she would like to make a swap on, 
    then he/she inputs an objective to make a compensation on
    
    Parameters
    __________
    df: dataframe
    max_df: max rank in dataframe before dropping dominated alternatives
    
    Returns
    _______
    None
    """

    print('\nThe first step is to choose an alternative')

    display(df)
    
    alternatives = df.columns[1:].values

    print('\nOptions are', alternatives)
    
    print('\nBelow are the alternative(s) that are likely to dominate another alternative')
    print('by having more objectives dominating the other alternative if any\n')
    
    dfwo = df.loc[:, df.columns[1:]] #Dataframe without objectives for counting ranks
    df_obj_count(dfwo)
    
    print('\nIt is better to make swaps on an alternative that is more likely to be dominated')
    print('\nBelow are the ranks that are almost equal if any\n')
    
    almost_equal(df)
    
    print('\nIt is better to make swaps on objectives where the ranks are almost equal')
    
    input_alt_text1 = '\nKnowing this information, which alternative would you' 
    input_alt_text2 = ' like to make an even swap on? Type quit to exit program: '
    input_alt_text = input_alt_text1 + input_alt_text2
    input_alt = input(input_alt_text)

    if (input_alt in alternatives):

        print('\nYou have chosen', input_alt)
        print('\nThe second step is to choose which objective you would like to make a swap on')

        objectives = df.iloc[:,0].values #Names of dataframe objectives
        print('\nOptions are', objectives)
        
        df_obj = df.iloc[:,0] # Dataframe objectives

        input_obj = input('\nChoose an objective to do an even swap on or type quit to exit program: ')
        obj_in_df = df_obj.isin([input_obj]).any() #Objective is in dataframe

        if (obj_in_df == True):
            chosen_obj = df.loc[df_obj == input_obj]
            display(chosen_obj)

            alt_value = chosen_obj[input_alt] #Alternative ranking value
            alt_value = int(alt_value.to_string(index = False))

            print('\nThe value for', input_obj, 'and', input_alt, 'is', alt_value)
            print('\nThe third step is to choose a new value for the objective and alternative')
            
            while True:
                try:
                    input_swap = int(input('\nWhat would you like the value to be changed to? '))

                except ValueError:
                    value_error_msg = '\nError! That is not an Int number. Please enter an Int number\n'
                    error_messages(value_error_msg)
                    continue
                else:
                    break

            print('\nYour chosen value is', input_swap)

            if (input_swap <= 0):
                error_msg1 = ('\nError! Your chosen value') 
                error_msg2 = (' is less than or equal to zero, but it has to be positive for the even swap\n')
                error_msg_whole = error_msg1 + error_msg2
                error_messages(error_msg_whole)

            elif (input_swap == alt_value):
                error_msg1 = ('\nError! Your input value cannot be the same as what the value for') 
                error_msg2 = (' the alternative already is, which is ') + str(alt_value)
                error_msg_whole = error_msg1 + error_msg2
                error_messages(error_msg_whole)

            elif (input_swap > max_df):
                error_msg1 = ('\nError! Your input value cannot be more than what the max value') 
                error_msg2 = (' in the dataframe is, which is ') + str(max_df)
                error_msg_whole = error_msg1 + error_msg2
                error_messages(error_msg_whole)

            else:
                df.loc[df_obj == input_obj, input_alt] = input_swap

                rem_obj = [x for x in objectives if x != input_obj] #Remaining objectives
                df_rem_obj = df.copy()
                df_rem_obj = df_rem_obj[df_rem_obj['Objectives'] != input_obj]

                if (len(objectives) == 2):
                    oth_obj = [x for x in rem_obj if x != input_obj] #Other objective
                    compensated_obj = oth_obj[0]
                else:
                    print('\nThe next step is to choose an objective to compensate for')
                    display(df_rem_obj)
                    print('\nOptions are', rem_obj)

                    compensated_obj = input('\nWhich objective would you like to compensate for?' +
                                            ' Type quit to exit program: ')

                comp_obj_in_df = df_obj.isin([compensated_obj]).any() #Compensated objective is in dataframe

                if (input_obj == compensated_obj):
                    error_msg1 = ('\nError! Your chosen compensated objective cannot be the same') 
                    error_msg2 = (' as the objective you would like to compensate for\n')
                    error_msg_whole = error_msg1 + error_msg2
                    error_messages(error_msg_whole)
                    df.loc[df_obj == input_obj, input_alt] = alt_value

                elif (comp_obj_in_df == True):
                    chosen_comp_obj = df.loc[df_obj == compensated_obj]
                    display(chosen_comp_obj)

                    alt_value_comp = chosen_comp_obj[input_alt]
                    alt_value_comp = int(alt_value_comp.to_string(index = False))

                    print('\nThe value for the compensated objective:', compensated_obj, end = ' ') 
                    print('and', input_alt, 'is', alt_value_comp)
                    print('\nThe next step is to choose a new compensated value for the objective and alternative')

                    while True:
                        try:
                            compensated_value = int(input('\nWhat would you like the compensated value to be? '))

                        except ValueError:
                            value_error_msg = '\nError! That is not an Int number. Please enter an Int number\n'
                            error_messages(value_error_msg)
                            df.loc[df_obj == input_obj, input_alt] = alt_value
                            continue
                        else:
                            break

                    print('\nYour chosen compensated value is', compensated_value)

                    if (compensated_value <= 0):
                        error_msg1 = ('\nError! Your chosen compensated value is less than or equal to zero,') 
                        error_msg2 = (' but the compensated value has to be positive for the even swap\n')
                        error_msg_whole = error_msg1 + error_msg2
                        error_messages(error_msg_whole)
                        df.loc[df_obj == input_obj, input_alt] = alt_value

                    elif (compensated_value == alt_value_comp):
                        error_msg1 = ('\nError! Your input value cannot be the same as what the value for') 
                        error_msg2 = (' the alternative already is, which is ') + str(alt_value_comp)
                        error_msg_whole = error_msg1 + error_msg2
                        error_messages(error_msg_whole)
                        df.loc[df_obj == input_obj, input_alt] = alt_value

                    elif (compensated_value > max_df):
                        error_msg1 = ('\nError! Your compensated value cannot be more than what the max value') 
                        error_msg2 = (' in the dataframe is, which is ') + str(max_df)
                        error_msg_whole = error_msg1 + error_msg2
                        error_messages(error_msg_whole)
                        df.loc[df_obj == input_obj, input_alt] = alt_value

                    else:
                        df.loc[df_obj == compensated_obj, input_alt] = compensated_value

                    print('\nDataframe after doing even swaps operations on it\n')
                    display(df)
                    print('\n****************\n')

                elif (compensated_obj.lower() == 'quit'):
                    raise EOFError

                else:
                    error_msg1 = ('\nError! ') + str(compensated_obj)
                    error_msg2 = (' is not part of the list of objectives\n')
                    error_msg_whole = error_msg1 + error_msg2
                    error_messages(error_msg_whole)
                    print('Options are', rem_obj, '\n')
                    df.loc[df_obj == input_obj, input_alt] = alt_value

        elif (input_obj.lower() == 'quit'):
            raise EOFError
        
        else:
            error_msg1 = ('\nError! ') + str(input_obj)
            error_msg2 = (' is not part of the list of objectives\n')
            error_msg_whole = error_msg1 + error_msg2
            error_messages(error_msg_whole)
            print('Options are', objectives, '\n')
            
    elif (input_alt.lower() == 'quit'):
        raise EOFError

    else:
        error_msg1 = ('\nError! ') + str(input_alt)
        error_msg2 = (' is not a valid name\n')
        error_msg_whole = error_msg1 + error_msg2
        error_messages(error_msg_whole)
        print('Options are', alternatives, '\n')

In [15]:
def dataframe_reduction (df):
    """
    Function that reduces a dataframe consisting of alternatives
    Uses the function compare_alternatives(alt1, alt2) to check for dominance
    If an alternative dominates another, the alternative that is 
    being dominated is removed from the dataframe
    
    Parameters
    __________
    df: dataframe to do the operations on
    
    Returns
    _______
    loser_count: counter to count how many losers there are
    """
    
    popped_obj = df.pop('Objectives')

    max_i = len(df.columns) - 1
    max_j = len(df.columns) - 1

    i = 0
    loser_count = 0

    while i <= max_i:

        j = 0
        alt1 = df.loc[:,df.columns[i]]

        while j <= max_j:

            alt2 = df.loc[:,df.columns[j]]
            dom_type, loser, winner = compare_alternatives(alt1, alt2)

            while loser in df:

                if (loser != 'none'):

                    df.drop(loser, axis = 1, inplace = True)
                    loser_count += 1

                    print()
                    print(loser, 'has been dropped from dataframe by the use of', dom_type, 'dominance.', end = ' ')
                    print('It is being dominated by', winner)
                    
                    if len(df.columns) > 1:
                        print('\nDisplaying new dataframe')
                        display(df)

                    if (loser == alt1.name):
                        max_i = len(df.columns) - 1
                        max_j = len(df.columns) - 1
                        j = max_j

                    else:
                        max_j -= 1
                        max_i -= 1
            else:
                j += 1

        if (loser != alt1.name):
            i += 1

    df.insert(0, 'Objectives', popped_obj)
    
    return loser_count

In [16]:
def running_even_swaps (df):
    """
    Reduces the dataframe when alternatives 
    are being dominated by using the function dataframe_reduction(df)
    Conducts even swaps to remove objectives
    
    Parameters
    __________
    df: dataframe to do the operations on
    
    Returns
    _______
    Returns if there are NaNs in the dataframe
    df: final dataframe after checking for dominance and conducting even swaps on it
    """

    rules = """
The following are the rules for using the program from this stage onwards:\n
1. Check for equal rank for the objectives, if True drops these objectives
2. Checks dominance, if alternative is being dominated (practical or absolute) it is dropped
3. User is asked to conduct even swaps on remaining alternatives and objectives
4. User selects one alternative he/she would like to make a swap on to make ranks equal
5. User is asked to compensate for this swap on another objective on same alternative
6. Final alternative should be the decision for the user
7. The user may quit on entering the even swaps part of the program by typing 'Quit' where prompted to
    """

    if df is None:
        print('\nThere is no dataframe to do even swaps on')
        
    else:
        print(rules)
        max_length = len(df.columns[1:])
        loser_count = 0

        while True:
            try:
                while max_length > 1:
                    popped_obj = df.pop('Objectives')
                    print('\nDataframe before removing equal rows')
                    display(df)
                    
                    isNan = df.isnull().values.any()
                    df = equality_check(df)
                    df.insert(0, "Objectives", popped_obj)
                    df.reset_index(drop = True, inplace = True)
                    
                    if isNan == True:
                        error_msg1 = ('\nError! Not possible to do even swaps as there are or could be') 
                        error_msg2 = (' NaNs in the dataframe. There is no final alternative')
                        error_msg_whole = error_msg1 + error_msg2
                        error_messages(error_msg_whole)
                        return
                    else:
                        print('\nDataframe after removing equal rows')
                        display(df)
                        
                        max_df = max_rank(df) #Max rank in DF before dominated alternatives are removed
                        loser_count = dataframe_reduction(df)
                        max_length -= loser_count

                        if (max_length > 1):
                            popped_obj = df.pop('Objectives')
                            print('\nDataframe before equal rows have been removed')
                            display(df)

                            df = equality_check(df)
                            df.insert(0, "Objectives", popped_obj)
                            df.reset_index(drop = True, inplace = True)
                            if isNan == True:
                                error_msg1 = ('\nError! Not possible to do even swaps as there are or could be') 
                                error_msg2 = (' NaNs in the dataframe. There is no final alternative')
                                error_msg_whole = error_msg1 + error_msg2
                                error_messages(error_msg_whole)
                                return
                            else:
                                print('\nDataframe after equal rows have been removed')
                                display(df)
                                even_swaps(df, max_df)
                return df

            except EOFError:
                print('\nProgram Quit')
                break

In [17]:
def load_files (filename):
    """
    Function to load a xlsx file into a dataframe
    
    Parameters
    __________
    filename: Name of the xlsx file to load
    
    Returns
    _______
    df: Loaded dataframe
    """
    
    df = pd.read_excel(filename)
    return df

In [18]:
def background_color (dat, c = 'red'):
    """
    Function to make the background color of a dataframe or data object red, or any other color
    
    Parameters
    __________
    dat: data object or df to change the background color on
    c: color, in this case it is set to be red
    
    Returns
    _______
    background color for dat
    """
    
    return [f'background-color: {c}' for i in dat]

In [19]:
def ES_process_and_examples ():
    """
    Function to describe the rules of the even swaps process
    and show an example of how it works
    
    Parameters
    __________
    None
    
    Returns
    _______
    None
    """
    
    rules = """
The following are the rules for conducting even swaps:\n
1. Create a consequences table (CT) of your alternatives and objectives as shown in fig. 1\n
2. Create a ranking table (RT) of the CT as shown in fig. 2
2a. This CT and RT are from figures in the Even Swaps article by Hammond et al. (1998)\n
3. Check for equal ranks on an objective in the RT, if equal ranks this objective 
can be removed as shown in fig. 3 and fig. 4\n
4. Check for absolute and practical dominance to eliminate alternatives
4a. Absolute dominance is when an alternative has better or equal rank on all objectives as shown in fig. 5
4b. Practical dominance is when an alternative has better or equal rank on all but one objective as shown in fig. 5
4c. For this example, Job B absolutely dominates Job E, this is shown in lime
4d. Jobs C and D are being practically dominated by Jobs B and A respectively, this is shown in magenta
4e. Job A is in cyan, while job B is in green. This is just to distinguish between the alternatives and 
the different types of dominance\n 
5. The remaining  alternatives and objectives are used for even swaps (fig. 6)
5a. Pick an alternative and an objective, e.g. in fig. 1, it could be alternative 'Job B' and objective 'MS'
5b. For this objective and alternative, change the rank such that it is 
equal to the rank of the other remaining alternatives (it is best to do this with alternatives where the rank is similar.
This makes it easier to make a swap.) This is shown in fig. 7
5c. For the same alternative, pick a different objective to make a compensation on, e.g. if you lowered the rank
in step 5b, you would increase the rank for the compensation objective shown in fig. 8
For this example the chosen objective to be swapped is opposite of what the compensation is.
It is easer to make swaps when the values are opposite for the swap and the compensation\n
6. Repeat steps 3 - 5 until a final alternative remains\n
    """
        
    print(rules)
    
    caption_style = [dict(selector = 'caption',
                          props = [('text-align', 'center'),
                                   ('font-size', '100%'),
                                   ('color', 'black')])]

    #Abbreviations
    abrvs = """ 
MS = Monthly Salary($)
FY = Flexibility
BSD = Business Skills Development
AVD = Annual Vacation Days
BS = Benefits
ET = Enjoyment
    """

    ct = pd.read_excel('Example Consequence and Ranking Table.xlsx', sheet_name = 'Sahid CT')
    ct_style = ct.style.set_caption('Fig. 1 Consequences Table').set_table_styles(caption_style)
    display(ct_style)

    rt = pd.read_excel('Example Consequence and Ranking Table.xlsx', sheet_name = 'Sahid RT')
    rt_style = rt.style.set_caption('Fig. 2 Ranking Table').set_table_styles(caption_style)
    display(rt_style)
    
    print('\nBelow are the abbreviations for the objectives for the CT and RT:')
    print(abrvs)

    er = pd.read_excel('Test DF Miller 1.xlsx') #Equal ranks

    er_copy = er.copy()
    highlight = lambda x: ['background: yellow' if x.name in [2] else '' for i in x]
    er_caption = 'Fig. 3 Equal ranks before removal'
    er = er.style.apply(highlight, axis = 1).set_caption(er_caption).set_table_styles(caption_style)

    display(er)

    er_copy.drop([2], axis = 0, inplace = True)
    er_copy_style = er_copy.style.set_caption('Fig. 4 Equal ranks after removal').set_table_styles(caption_style)
    display(er_copy_style)

    rt_copy = rt.copy()

    columns_with_color_dictionary = {'Job E': 'lime', 'Job B': 'green', 'Job D': 'magenta',
                                    'Job C': 'magenta', 'Job A': 'cyan'}

    style = rt_copy.style.set_caption('Fig. 5 Absolute and Practical dominance').set_table_styles(caption_style)
    for column, color in columns_with_color_dictionary.items():
        style = style.apply(background_color, axis = 0, subset = column, c = color)
    display(style)

    sahid_caption = 'Fig. 6 Example Ranking Table after removing dominated alternatives'
    sahid_rem_alts = pd.read_excel('Test RT Sahid.xlsx')
    highlight_sahid = lambda x: ['background: yellow' if x.name in [0, 5] else '' for i in x]
    sahid_rem_alts = sahid_rem_alts.style.apply(
        highlight_sahid, axis = 1).set_caption(sahid_caption).set_table_styles(caption_style)

    display(sahid_rem_alts)

    sahid_caption2 = 'Fig. 7 Example Ranking Table after doing an even swap'
    sahid_rem_alts2 = pd.read_excel('Test RT Sahid 2.xlsx')

    sahid_rem_alts_copy = sahid_rem_alts2.copy()

    highlight_sahid2 = lambda x: ['background: yellow' if x.name in [0, 5] else '' for i in x]
    sahid_rem_alts2 = sahid_rem_alts2.style.apply(
        highlight_sahid2, axis = 1).set_caption(sahid_caption2).set_table_styles(caption_style)

    display(sahid_rem_alts2)

    sahid_rem_alts_copy.drop([0, 5], axis = 0, inplace = True)
    sahid_rem_alts_copy_style = sahid_rem_alts_copy.style.set_caption(
        'Fig. 8 Example Ranking table after dropping equal ranks').set_table_styles(caption_style)
    display(sahid_rem_alts_copy_style)

In [20]:
def readUsers():
    """
    Function to read users from a json file
    
    Parameters
    __________
    None
    
    Returns
    _______
    {}: empty dictionary if not able to find the file
    """
    try:
        with open('users_file.json', 'r') as f: #users.json
            return json.load(f)
    except FileNotFoundError:
        return {}

In [21]:
def writeUsers(usr):
    """
    Function to write users to the json file
    
    Parameters
    __________
    None
    
    Returns
    _______
    None
    """
    with open('users_file.json', 'w+') as f: #users.json
            json.dump(usr, f)

In [22]:
def login(usr):
    """
    Function to login a user to use the even swaps program by asking for username and password
    
    Parameters
    __________
    usr: users are read in from the function readUsers()
    
    Returns
    _______
    returns False if the password is incorrect or if the user enters quit
    returns True if able to write user to file by using function writeUsers()
    """
    global uN
    global pW
    flag = True
    while flag:
        uN = input('\nEnter your username or enter quit to quit the program: ') #username
        if uN.lower() == 'quit':
            print('\nProgram quit')
            break
            return False
        pW = input('\nEnter your password or enter quit to quit the program: ') #password
        if pW.lower() == 'quit':
            print('\nProgram quit')
            break
            return False
        if uN in usr.keys():
            if pW == usr[uN]:
                print('\nWelcome back ' + uN + '!')
                flag = False
            else:
                error_msg = ('\nIncorrect password for user: ' + uN)
                error_messages(error_msg)
                return False
        else:
            if uN.lower() == 'quit':
                print('Username is', uN)
                break
                return False
            elif pW.lower() == 'quit':
                print('Password is', pW)
                break
                return False
            else:
                decision = input('\nAre you happy with your username and password? Y or N: ')
                if decision.lower() in {'y'}:
                    print('\nWelcome to the even swaps program ' + uN + '!')
                    usr[uN] = pW
                    ES_process_and_examples()

                    uflag = True #understanding flag
                    while uflag:
                        understanding = input('\nDo you understand the even swaps process? Y or N: ')

                        if understanding.lower() in {'n'}:
                            ES_process_and_examples()
                        elif understanding.lower() in {'y'}:
                            uflag = False
                            flag = False
                        else:
                            error_msg = '\nError! Options are Y or N'
                            error_messages(error_msg)

                elif decision.lower() in {'n'}:
                    uN = input('\nEnter your username or enter quit to quit the program: ') #username
                    pW = input('\nEnter your password or enter quit to quit the program: ') #password
                    if uN in usr.keys():
                        if pW == usr[uN]:
                            print('\nWelcome back ' + uN + '!')
                            flag = False
                        else:
                            print('\nIncorrect password for user:', uN)
                            return False
                    else:
                        print('\nWelcome to the even swaps program ' + uN + '!')
                        usr[uN] = pW
                        ES_process_and_examples()

                        uflag = True #understanding flag
                        while uflag:
                            understanding = input('\nDo you understand the even swaps process? Y or N: ')

                            if understanding.lower() in {'n'}:
                                ES_process_and_examples()
                            elif understanding.lower() in {'y'}:
                                uflag = False
                                flag = False
                            else:
                                error_msg = '\nError! Options are Y or N'
                                error_messages(error_msg)
                else:
                    error_msg = '\nError! Options are Y or N'
                    error_messages(error_msg)

    writeUsers(usr)
    return True

In [25]:
def ES_and_file_creation ():
    """
    Function that executes all other functions, and asks the user whether he/she would like
    to create a csv file of the final dataframe
    
    Parameters
    __________
    None
    
    Returns
    _______
    return_msg: Returns message if there is no final dataframe
    """
    
    global uN
    global pW
    
    #esp: Even swap program
    esp_rules = """ 
The following are the rules for using the even swaps program:\n
1. You are asked to input your username and password
1a. If new user, you will input what you want your username and password to be
1b. Then, the rules for conducting even swaps are displayed for you and you will go through an even swaps example
1c. If returning user, you will be asked whether you would like to create a consequences table (CT) and ranking table (RT)
or if you would like to load a CT or RT from a file
2. You are asked to create an RT if you created a CT in the program or if you loaded a CT from a file
3. Once in RT format, the program checks for equal attributes and removes these and checks for dominance 
removing any dominated alternative
4. The program asks whether you would like to make an even swap and displays whether an alternative is likely 
to dominate another, and displays the objective ranks that are almost equal such that it is easier to see what
objectives and alternatives you should make swaps on
    """
    
    print(esp_rules)
    
    user_info_flag = True
    flag = True
    while user_info_flag:
        try:           
            users = readUsers()
            success = login(users)
            if uN.lower() == 'quit' or pW.lower() == 'quit':
                break
            else:
                while not success:
                    success = login(users)
                while flag:
                    try:
                        input_decision = input('\nWould you like to Create Your Own (CYO) or Load a Created File (LCF)?' + 
                                                ' (Options are CYO or LCF) Enter quit to quit: ')
                        if input_decision.lower() == 'quit':
                            user_info_flag = False

                        df = user_choice(input_decision)
                    except EOFError:
                        print('\nProgram Quit')
                        break

                    final_df = running_even_swaps(df)

                    return_msg = ''

                    if final_df is None:
                        return_msg = "There is no final dataframe"
                        print()
                        return return_msg

                    else:
                        print('\nThis is the final alternative')

                        display(final_df)
    
                        ifd_text = '\nWould you like to create a file of the final dataframe? Y or N: '
                        input_filename_decision = input(ifd_text)
                        if input_filename_decision.lower() in {'y'}:

                            input_filename = input('\nEnter the name of your csv file: ') + '.csv'
                            print()
                            print(input_filename)

                            final_df.to_csv(input_filename)
                            read_df = pd.read_csv(input_filename)
                            read_df.drop(read_df.iloc[:,0:1], inplace = True, axis = 1)
                            print('\nDisplaying dataframe of final alternative file')
                            display(read_df)
                            flag = True

                        elif input_filename_decision.lower() in {'n'}:
                            flag = True

                        else:
                            error_msg = '\nError! Options are Y or N'
                            error_messages(error_msg)
        except EOFError:
            print('\nProgram quit')
            user_info_flag = False
            flag = False
    
es_func = ES_and_file_creation()
print(es_func)

 
The following are the rules for using the even swaps program:

1. You are asked to input your username and password
1a. If new user, you will input what you want your username and password to be
1b. Then, the rules for conducting even swaps are displayed for you and you will go through an even swaps example
1c. If returning user, you will be asked whether you would like to create a consequences table (CT) and ranking table (RT)
or if you would like to load a CT or RT from a file
2. You are asked to create an RT if you created a CT in the program or if you loaded a CT from a file
3. Once in RT format, the program checks for equal attributes and removes these and checks for dominance 
removing any dominated alternative
4. The program asks whether you would like to make an even swap and displays whether an alternative is likely 
to dominate another, and displays the objective ranks that are almost equal such that it is easier to see what
objectives and alternatives you should make swaps

,Objectives,Parkway,Lombard,Baranov,Montana,Pierpoint
0,CIM,5,2,1,2,4
1,CA,5,2,4,1,3
2,OS,1,3,4,1,4
3,OSSF,2,3,5,1,3
4,MC,4,2,1,5,3



The following are the rules for using the program from this stage onwards:

1. Check for equal rank for the objectives, if True drops these objectives
2. Checks dominance, if alternative is being dominated (practical or absolute) it is dropped
3. User is asked to conduct even swaps on remaining alternatives and objectives
4. User selects one alternative he/she would like to make a swap on to make ranks equal
5. User is asked to compensate for this swap on another objective on same alternative
6. Final alternative should be the decision for the user
7. The user may quit on entering the even swaps part of the program by typing 'Quit' where prompted to
    

Dataframe before removing equal rows


,Parkway,Lombard,Baranov,Montana,Pierpoint
0,5,2,1,2,4
1,5,2,4,1,3
2,1,3,4,1,4
3,2,3,5,1,3
4,4,2,1,5,3



Dataframe after removing equal rows


...


Dataframe after equal rows have been removed


,Objectives,Baranov,Montana
0,CIM,1,2
1,CA,4,1
2,OS,4,1
3,OSSF,5,1
4,MC,1,5



The first step is to choose an alternative


,Objectives,Baranov,Montana
0,CIM,1,2
1,CA,4,1
2,OS,4,1
3,OSSF,5,1
4,MC,1,5



Options are ['Baranov' 'Montana']

Below are the alternative(s) that are likely to dominate another alternative
by having more objectives dominating the other alternative if any

Montana is more likely to dominate Baranov

It is better to make swaps on an alternative that is more likely to be dominated

Below are the ranks that are almost equal if any

Objective CIM for Baranov and Montana is almost equal with these rankings: 1, 2

It is better to make swaps on objectives where the ranks are almost equal

Knowing this information, which alternative would you like to make an even swap on? Type quit to exit program: quit

Program Quit

There is no final dataframe
